# Alice

* Alice wants to send a message to Bob.
* She can communicate with Bob over an unsecure, classical channel.
* They can also communicate over a quantum channel.

In [ ]:
# Raspberry Pi exclusive imports:
from relay_lib_seeed import *  # to control the laser relay.

Alice establishes a classical channel to Bob. Anyone can listen in:

In [ ]:
# server = "broker.emqx.io"  # fallback server.
mqtt_broker_ip = "localhost"

channel = "quantum_cryptography/classical_channel"

import paho.mqtt.client as mqtt

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))


def on_publish(client, userdata, result):
    print("message published")


client = mqtt.Client()
client.on_connect = on_connect
client.on_publish = on_publish

client.connect(mqtt_broker_ip, 1883, 60)

**Step 2**: Alice sends a message to Bob:

In [ ]:
plaintext = input()
_ = client.publish(channel, payload=plaintext, qos=0, retain=False)